In [ ]:
!nvidia-smi

## Install Ollama

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

In [5]:
!pip install -qq pyngrok ollama

## Install Ollama Model

In [ ]:
!ollama pull nomic-embed-text

In [ ]:
!ollama pull phi3:3.8b

## Dependencies

In [6]:
import IPython
import subprocess
from pyngrok import ngrok
from google.colab import userdata
from typing import Any, Dict, List, Optional

## Auxiliary Functions

In [10]:
def start_ollama_server() -> None:
  """Starts the Ollama server."""
  subprocess.Popen(['ollama', 'serve'])
  print("Ollama server started.")

def check_ollama_port(port: str) -> None:
  """Check if Ollama server is running at the specified port."""
  try:
      subprocess.run(['sudo', 'lsof', '-i', '-P', '-n'], check=True, capture_output=True, text=True)
      if any(f":{port} (LISTEN)" in line for line in subprocess.run(['sudo', 'lsof', '-i', '-P', '-n'], capture_output=True, text=True).stdout.splitlines()):
        print(f"Ollama is listening on port {port}")
      else:
        print(f"Ollama does not appear to be listening on port {port}.")
  except subprocess.CalledProcessError as e:
      print(f"Error checking Ollama port: {e}")

def setup_ngrox_tunnel(port: str) -> ngrok.NgrokTunnel:
  """Sets up an ngrok tunnel.

  Args:
    port: Th port to tunnel.

  Returns:
    The ngrok tunnel object.

  Raises:
    RuntimeError: If the ngrok authentoken is not set.
  """
  ngrok_auth_token = userdata.get('NGROK_AUTHTOKEN')
  if not ngrok_auth_token:
      raise RuntimeError("NGROK_AUTHTOKEN is not set.")

  ngrok.set_auth_token(ngrok_auth_token)
  tunnel = ngrok.connect(port, host_header=f'localhost:{port}')
  print(f"ngrok tunnel created: {tunnel.public_url}")
  return tunnel

## Parameters

In [8]:
NGROK_PORT = '11434'

# Start the `Ollama` server

In [11]:
start_ollama_server()

check_ollama_port(NGROK_PORT)

Ollama server started.
Ollama is listening on port 11434


## Setup ngrok `ngrok` tunnel

In [ ]:
ngrok_tunnel = setup_ngrox_tunnel(NGROK_PORT)